In [4]:
import pandas as pd
import numpy as np
from scipy.stats import f as fisher_f

In [6]:
#loading data
df = pd.read_csv("/content/drive/MyDrive/Data Science Eugene Pinsky/Homework_9/NVDA.csv")
df.head()

,Date,Year,Month,Day,Weekday,Week_Number,Year_Week,Open,High,Low,Close,Volume,Adj Close,Return,Short_MA,Long_MA
0,2017-01-03 00:00:00-05:00,2017,1,3,Tuesday,1,2017-01,26.10,26.59,24.84,25.50,150199600,25.15,0.000000,25.150000,25.150000
1,2017-01-04 00:00:00-05:00,2017,1,4,Wednesday,1,2017-01,25.85,26.38,25.38,26.10,119922000,25.74,0.023331,25.445000,25.445000
2,2017-01-05 00:00:00-05:00,2017,1,5,Thursday,1,2017-01,26.13,26.45,25.26,25.43,98429600,25.09,-0.025385,25.326667,25.326667
3,2017-01-06 00:00:00-05:00,2017,1,6,Friday,1,2017-01,25.71,26.06,25.30,25.77,82285600,25.42,0.013367,25.350000,25.350000
4,2017-01-09 00:00:00-05:00,2017,1,9,Monday,2,2017-02,25.88,27.00,25.88,26.82,91624800,26.45,0.040543,25.570000,25.570000


In [7]:
#getting data for each year
year_1_data = df[df["Year"] == 2020]
year_1_data = year_1_data.reset_index(drop=True)

year_2_data = df[df["Year"] == 2021]
year_2_data = year_2_data.reset_index(drop=True)

Function to compute candidate days along with SSE for both both regression models for each month of the year

In [8]:
def compute_candidate_by_year(year_data):

  #dataframe that stores the best possible single candidate date for each month of the year
  candidate_across_months = pd.DataFrame(columns=["K","Month","SSE_1","SSE_2","SSE_1 + SSE_2"])

  #number of days
  n = 19

  #for every month 
  for month in range(1,13):

    residuals_1 = 0 
    
    residuals_2 = 0

    SSE_1 = 0

    SSE_2 = 0

    #temporary candidate month dataframe to store all possible candidate dates for a month and is reset before computing candidate for every month
    candidate_by_month = pd.DataFrame(columns=["K","Month","SSE_1","SSE_2","SSE_1 + SSE_2"])

    #for each size of sliding window from 2 to n-2 days
    for i in range (2,n-1):

      #getting data for each month
        month_data = year_data[year_data["Month"] == month]

        #getting segment 1
        T1 = month_data.iloc[:i]

        #computing regression model based on first segment
        reg1, residuals_1, rank, singular_values, rcond = np.polyfit(T1["Day"],T1["Adj Close"],1,full=True)

        #getting segment 2
        T2 = month_data.iloc[i+1:]

        #computing regression model based on second segment
        reg2, residuals_2, rank, singular_values, rcond = np.polyfit(T2["Day"],T2["Adj Close"],1,full=True)

        #computed SSE by adding additional parameter full=True in polyfit function

        #code to prevent 0 axis error i.e if array computed is empty
        if residuals_1.size != 0:
          SSE_1 = residuals_1[0]

        if residuals_2.size != 0:
          SSE_2 = residuals_2[0]

        #adding data for each month under each window size
        candidate_by_month = pd.concat( [ candidate_by_month , pd.DataFrame([[i,month,SSE_1,SSE_2,SSE_1 + SSE_2 ]],columns=["K","Month","SSE_1","SSE_2","SSE_1 + SSE_2"]) ] )

    #retrieving candidate date with lowest sum of losses
    candidate_record = candidate_by_month[candidate_by_month["SSE_1 + SSE_2"] == candidate_by_month["SSE_1 + SSE_2"].min()]

    #storing candidates for each month 
    candidate_across_months = pd.concat([candidate_across_months,candidate_record])

  return candidate_across_months

Function to compute SSE for simple linear regression over the whole of each month of the year

In [9]:
def compute_linear_reg_by_year(year_data):

  #dataframe that stores the best possible single candidate date for each month of the year
  candidate_across_months = pd.DataFrame(columns=["Month","SSE"])

  #number of days
  n = 19

  #for every month 
  for month in range(1,13):

    residuals_1 = 0 
    
    residuals_2 = 0

    SSE_1 = 0

    SSE_2 = 0

    #getting data for each month
    month_data = year_data[year_data["Month"] == month]

    #computing regression model based on first segment
    reg1, residuals_1, rank, singular_values, rcond = np.polyfit(month_data["Day"],month_data["Adj Close"],1,full=True)

    #computed SSE by adding additional parameter full=True in polyfit function
    #code to prevent 0 axis error i.e if array computed is empty
    if residuals_1.size != 0:
      SSE_1 = residuals_1[0]

    #adding data for each month under each window size
    candidate_across_months = pd.concat( [ candidate_across_months , pd.DataFrame([[month,SSE_1]],columns=["Month","SSE"]) ] )

  return candidate_across_months

function to construct f distribution table

In [2]:
def construct_F_distribution(month_data,L_data,L1_plus_L2,n):

  #creating table to get required values to compute row wise f statistics
  f_statistic_table = pd.DataFrame(columns=["Month","L","L1+L2"])

  f_statistic_table["Month"] = month_data

  f_statistic_table["L"] = L_data

  f_statistic_table["L1+L2"] = L1_plus_L2

  f_statistic_table = f_statistic_table.reset_index(drop=True)

  f_statistic_table["f"] = ((f_statistic_table["L"] - f_statistic_table["L1+L2"])/2) * ((n-4)/f_statistic_table["L1+L2"])
  
  #computing F distribution 
  p_value_1 = fisher_f.cdf(f_statistic_table["f"],2,n-4)

  F_distribution = pd.DataFrame(columns=["Month","p_value"])
  
  F_distribution["Month"] = simple_reg_across_months_year_1["Month"]
  
  F_distribution["p_value"] = p_value_1
  
  return F_distribution

##Q1.

### Computing F statistics for year 1.

Getting results of simple linear regression using time period as T (whole month) that has loss SSE.

In [10]:
#getting table of SSE for each month by doing linear regression 
simple_reg_across_months_year_1= compute_linear_reg_by_year(year_1_data)
simple_reg_across_months_year_1

,Month,SSE
0,1,28.655128
0,2,395.777191
0,3,843.165624
0,4,82.326332
0,5,160.138407
0,6,47.525615
0,7,93.126777
0,8,127.287368
0,9,688.084566
0,10,304.255943


Gettting results of computing candidate day for each month using two two time periods T1 and T2 having two regression models with losses SSE_1 and SSE_2 respectively.

In [11]:
candidate_across_months_year_1= compute_candidate_by_year(year_1_data)
candidate_across_months_year_1

,K,Month,SSE_1,SSE_2,SSE_1 + SSE_2
0,16,1,5.842837,1.231220,7.074057
0,13,2,20.938426,14.741830,35.680256
0,15,3,108.086449,3.740615,111.827063
0,13,4,30.070049,4.164435,34.234484
0,10,5,15.524081,20.285169,35.809249
0,7,6,3.690791,29.516493,33.207284
0,7,7,12.387287,26.159533,38.546820
0,6,8,6.07463,44.143910,50.218540
0,2,9,0,215.126009,215.126009
0,6,10,22.69464,44.211470,66.906110


Creating F distribution using construct F distribution function.

In [13]:
F_dist = construct_F_distribution(simple_reg_across_months_year_1["Month"],simple_reg_across_months_year_1["SSE"],candidate_across_months_year_1["SSE_1 + SSE_2"],19)
F_dist

,Month,p_value
0,1,0.999972
0,2,1.000000
0,3,1.000000
0,4,0.998613
0,5,0.999987
0,6,0.932034
0,7,0.998661
0,8,0.999065
0,9,0.999837
0,10,0.999988


### Computing F statistics for Year 2 

In [14]:
#getting table of SSE for each month by doing linear regression 
simple_reg_across_months_year_2= compute_linear_reg_by_year(year_2_data)
simple_reg_across_months_year_2

,Month,SSE
0,1,191.295224
0,2,754.434701
0,3,458.518496
0,4,531.708991
0,5,602.068899
0,6,165.973826
0,7,606.496584
0,8,1231.310606
0,9,357.800547
0,10,740.801036


In [15]:
candidate_across_months_year_2= compute_candidate_by_year(year_2_data)
candidate_across_months_year_2

,K,Month,SSE_1,SSE_2,SSE_1 + SSE_2
0,11,1,92.372348,14.845285,107.217633
0,9,2,33.247022,61.167811,94.414833
0,6,3,22.482492,113.467070,135.949562
0,6,4,7.557997,103.062831,110.620829
0,9,5,45.456841,49.921893,95.378734
0,4,6,2.95623,88.074155,91.030385
0,9,7,58.292493,101.686396,159.978889
0,14,8,150.878414,28.355983,179.234398
0,14,9,113.404086,35.380834,148.784921
0,17,10,233.821555,0.308267,234.129822


In [16]:
F_dist = construct_F_distribution(simple_reg_across_months_year_2["Month"],simple_reg_across_months_year_2["SSE"],candidate_across_months_year_2["SSE_1 + SSE_2"],19)
F_dist

,Month,p_value
0,1,0.986992
0,2,1.000000
0,3,0.999890
0,4,0.999992
0,5,0.999999
0,6,0.988944
0,7,0.999954
0,8,0.999999
0,9,0.998614
0,10,0.999823
